# Homework 1

## FINM 37500 - 2023

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

# Context

For use in these problems, consider the data below, discussed in Veronesi's *Fixed Income Securities* Chapters 9, 10.
* interest-rate tree
* current term structure

In [1]:
import numpy as np
import pandas as pd
from scipy import optimize

In [2]:
rate_tree = pd.DataFrame({'0':[.0174,np.nan],'0.5':[.0339,.0095]})
rate_tree.columns.name = 'time $t$'
rate_tree.index.name = 'node'
rate_tree.style.format('{:.2%}',na_rep='')

time $t$,0,0.5
node,,
0,1.74%,3.39%
1,,0.95%


The "tree" is displayed as a pandas dataframe, so it does not list "up" and "down" for the rows but rather an index of nodes. The meaning should be clear.

In [3]:
term_struct = pd.DataFrame({'maturity':[.5,1,1.5],'price':[99.1338,97.8925,96.1462]})
term_struct['continuous ytm'] = -np.log(term_struct['price']/100) / term_struct['maturity']
term_struct.set_index('maturity',inplace=True)
term_struct.style.format({'price':'{:.4f}','continuous ytm':'{:.2%}'}).format_index('{:.1f}')

,price,continuous ytm
maturity,,
0.5,99.1338,1.74%
1.0,97.8925,2.13%
1.5,96.1462,2.62%


This is the current term-structure observed at $t=0$.

# 1. Pricing a Swap

### 1.1 
Calculate the tree of bond prices for the 2-period, $T=1$, bond.

### 1.2 
What is the risk-neutral probability of an upward movement of interest rates at $t=.5$?

## The option contract

Consider a single-period swap that pays at time period 1 ($t=0.5$), the expiration payoff (and thus terminal value) is
* Payoff = $\frac{100}{2}(r_1 −c)$
* with $c=2\%$
* payments are semiannual

Take the viewpoint of a fixed-rate payer, floating rate receiver.

### 1.3 
What is the replicating trade using the two bonds (period 1 and period 2)?

### 1.4 
What is the price of the swap?

In [4]:
print(1.1)
bond_tree = pd.DataFrame({'0':[term_struct.loc[1.0,"price"],np.nan],'0.5':[100*np.exp(-0.5*rate_tree.loc[0,'0.5']),100*np.exp(-0.5*rate_tree.loc[1,'0.5'])]})
bond_tree.columns.name = 'time $t$'
bond_tree.index.name = 'node'
bond_tree.style.format('{:.4f}',na_rep='')

1.1


time $t$,0,0.5
node,,
0,97.8925,98.3193
1,,99.5261


In [17]:
print(1.2)
up_pv = bond_tree.loc[0,'0.5'] * np.exp(-0.5 * rate_tree.loc[0,'0'])
down_pv = bond_tree.loc[1,'0.5'] * np.exp(-0.5 * rate_tree.loc[0,'0'])
p_up = (bond_tree.loc[0,'0']-down_pv)/(up_pv -down_pv)
'risk neutral probability of upward rate movement is ' + format(p_up,'.4f')

1.2


'risk neutral probability of upward rate movement is 0.6449'

In [19]:
print('1.3 & 1.4')
swap_payoff_up =100/2*( rate_tree.loc[0,'0.5'] - 0.02)
swap_payoff_down =100/2*( rate_tree.loc[1,'0.5'] - 0.02)
#print(swap_payoff_up, swap_payoff_down)


bond_payoff_up = bond_tree.loc[0,'0.5']
bond_payoff_down = bond_tree.loc[1,'0.5']
#print(bond_payoff_up, bond_payoff_down)

beta = (swap_payoff_up-swap_payoff_down)/(bond_payoff_up-bond_payoff_down)
pos_1y = beta
pos_6m = (swap_payoff_up - beta * bond_tree.loc[0,'0.5'])/100
cost_of_replicating_pos = pos_1y * term_struct.loc[1.0,'price']+ pos_6m * term_struct.loc[0.5,'price']
#print(beta)

print("To replicate a contract of the swap, we need to long ", format(pos_6m,".4f"), " bonds with 6m maturity and short ", format(-pos_1y,".4f"), "bonds with 1y maturity (each bond is $100 face value)" )

#print(pos_1y * term_struct.loc[1.0,'price']+ pos_6m * term_struct.loc[0.5,'price'])
#print(pos_1y * bond_tree.loc[0,'0.5'] + pos_6m * 100)
#print(pos_1y * bond_tree.loc[1,'0.5'] + pos_6m * 100)

print("The price of the swap must equal the cost of setting up the replicating trade, which is $" + format(cost_of_replicating_pos,".4f"))

#(swap_payoff_up * p_up +swap_payoff_down * (1-p_up))*np.exp(-0.5*0.0174)

1.3 & 1.4
To replicate a contract of the swap, we need to long  1.0009  bonds with 6m maturity and short  1.0109 bonds with 1y maturity (each bond is $100 face value)
The price of the swap must equal the cost of setting up the replicating trade, which is $0.2595


# 2. Using the Swap as the Underlying
As in the note, W.1, consider pricing the followign interest-rate option,
* Payoff is $100\max(r_K-r_1,0)$
* strike is $r_K$ is 2\%
* expires at period 1, ($t=0.5$) 

Unlike the note, price it with the swap used as the underlying, not the two-period ($t=1$) bond. You will once again use the period-1 ($t=0.5$) bond as the cash account for the no-arbitrage pricing.

So instead of replicating the option with the two treasuries, now you're replicating/pricing it with a one-period bond and two-period swap.

### 2.1
Display the tree of swap prices.

### 2.2
What is the risk-neutral probability of an upward movement at $t=.5$ implied by the underlying swap tree? 

Is this the same as the risk-neutral probability we found when the bond was used as the underlying?

### 2.3
What is the price of the rate option? Is it the same as we calculated in the note, W.1.?

In [7]:
swap_tree = pd.DataFrame({'0':[cost_of_replicating_pos,np.nan],'0.5':[swap_payoff_up,swap_payoff_down]})
swap_tree.columns.name = 'time $t$'
swap_tree.index.name = 'node'
swap_tree.style.format('{:.4f}',na_rep='')
print(2.1)
print(swap_tree)

2.1
time $t$        0    0.5
node                    
0         0.25949  0.695
1             NaN -0.525


In [8]:
p_up_swap = (cost_of_replicating_pos*np.exp(rate_tree.loc[0,'0']*0.5)-swap_payoff_down)/(swap_payoff_up-swap_payoff_down)
print(2.2)
print('risk neutral probability of upward rate movement implied by the swap is ' + format(p_up,'.2f'))
print('same as the probability from underlying')

2.2
risk neutral probability of upward rate movement implied by the swap is 0.64
same as the probability from underlying


In [20]:
print('2.3')
option_payoff_up =0
option_payoff_down =100*(0.02 - rate_tree.loc[1,'0.5'] )
#print(option_payoff_up, option_payoff_down)



beta_swap = (option_payoff_up-option_payoff_down)/(swap_payoff_up-swap_payoff_down)
pos_swap = beta_swap
pos_6m = (option_payoff_up - beta_swap * swap_payoff_up)/100
cost_of_replicating_option = pos_swap * cost_of_replicating_pos + pos_6m * term_struct.loc[0.5,'price']
#print(beta)

print("To replicate a contract of the option, we need to long ", format(pos_6m,".5f"), " bonds with 6m maturity and short ", format(-pos_swap,".4f"), "swaps (each bond is $100 face value)" )

#print(pos_swap *swap_payoff_up + pos_6m * 100)
#print(pos_swap * swap_payoff_down + pos_6m * 100)


print("The price of the option must equal the cost of setting up the replicating trade, which is $" + format(cost_of_replicating_option,".4f") +", same as calculated in class.")

#(option_payoff_up * p_up +option_payoff_down * (1-p_up))*np.exp(-0.5*0.0174)

2.3
To replicate a contract of the option, we need to long  0.00598  bonds with 6m maturity and short  0.8607 swaps (each bond is $100 face value)
The price of the option must equal the cost of setting up the replicating trade, which is $0.3696, same as calculated in class.


# 3. Pricing a Call on a Bond

Try using the same tree to price a call on the period-2 bond, (1-year), at period 1 (6-months).
* Payoff = $\max(P_{1|2}-K,0)$
* Strike = \$99.00

### 3.1 
What is the replicating trade using the two bonds (period 1 and period 2) as above? (That is, we are no longer using the swap as the underlying.)

### 3.2 
What is the price of the European call option? 
* expiring at $T=.5$ 
* written on the bond maturing in 2 periods, ($t=1$)

In [21]:
print('3.1 & 3.2')
bond_option_payoff_up =max(bond_payoff_up-99,0)
bond_option_payoff_down =max(bond_payoff_down-99,0)
#print(bond_option_payoff_up, bond_option_payoff_down)



beta = (bond_option_payoff_up-bond_option_payoff_down)/(bond_payoff_up-bond_payoff_down)
pos_1y = beta
pos_6m = (bond_option_payoff_up - beta * bond_tree.loc[0,'0.5'])/100
cost_of_replicating_bond_option = pos_1y * term_struct.loc[1.0,'price']+ pos_6m * term_struct.loc[0.5,'price']
#print(beta)

print("To replicate a contract of the bond option, we need to short ", format(-pos_6m,".4f"), " bonds with 6m maturity and long ", format(pos_1y,".4f"), "bonds with 1y maturity (each bond is $100 face value)" )

#print(pos_1y * term_struct.loc[1.0,'price']+ pos_6m * term_struct.loc[0.5,'price'])
#print(pos_1y * bond_tree.loc[0,'0.5'] + pos_6m * 100)
#print(pos_1y * bond_tree.loc[1,'0.5'] + pos_6m * 100)

print("The price of the bond option must equal the cost of setting up the replicating trade, which is $" + format(cost_of_replicating_bond_option,".4f"))

#(bond_option_payoff_up * p_up +bond_option_payoff_down * (1-p_up))*np.exp(-0.5*0.0174)

3.1 & 3.2
To replicate a contract of the bond option, we need to short  0.4286  bonds with 6m maturity and long  0.4360 bonds with 1y maturity (each bond is $100 face value)
The price of the bond option must equal the cost of setting up the replicating trade, which is $0.1852


# 4 Two-Period Tree

Consider an expanded, **2 period** tree. (Two periods of uncertainty, so with the starting point, three periods total.)

In [11]:
new_col = pd.Series([.05,.0256,.0011],name='1')
rate_tree_multi = pd.concat([rate_tree,new_col],ignore_index=True,axis=1)
rate_tree_multi.columns = pd.Series(['0','0.5','1'],name='time $t$')
rate_tree_multi.index.name = 'node'
rate_tree_multi.style.format('{:.2%}',na_rep='')

time $t$,0,0.5,1
node,,,
0,1.74%,3.39%,5.00%
1,,0.95%,2.56%
2,,,0.11%


### 4.1

Calculate and show the tree of prices for the 3-period bond, $T=1.5$.

### 4.2
Report the risk-neutral probability of an up movement at $t=1$.

(The risk-neutral probability of an up movement at $t=0.5$ continues to be as you calculated in 2.3.

### 4.3
Calculate the price of the European **call** option?
* expiring at $T=1$ 
* written on the bond maturing in 3 periods, ($t=1.5$)

### 4.4
Consider a finer time grid. Let $dt$ in the tree now be 1/30 instead of 0.5.

Using this smaller time step, compute the $t=0$ price of the following option:
* option expires at $t=1$
* written on bond maturing at $t=1.5

In [12]:
def start_price(end_price, continuous_rate, period):
    return end_price * np.exp(-period * continuous_rate)

In [13]:
price_t1 = start_price(100,rate_tree_multi["1"],0.5)
def pv0_bond18m(p_up_t1):
    return price_t1[0] * np.exp(-0.5* (0.0339+0.0174)) * p_up_t1 * p_up + price_t1[1] * np.exp(-0.5* (0.0339+0.0174)) * (1-p_up_t1)*p_up + price_t1[1] * np.exp(-0.5* (0.0095+0.0174)) * (p_up_t1)*(1-p_up) +  price_t1[2] * np.exp(-0.5* (0.0095+0.0174)) * (1-p_up_t1)*(1-p_up)

def target_func(p_up_t1,target):
    return pv0_bond18m(p_up_t1) - target

p_up_t1_root = optimize.fsolve(target_func,0,args=(term_struct.loc[1.5,'price']))[0]

bond_18m_u = price_t1[0] * np.exp(-0.5* (0.0339)) * p_up_t1_root  + price_t1[1] * np.exp(-0.5* (0.0339)) * (1-p_up_t1_root)
bond_18m_d = price_t1[1] * np.exp(-0.5* (0.0095)) * p_up_t1_root  + price_t1[2] * np.exp(-0.5* (0.0095)) * (1-p_up_t1_root)

price_t6m = [bond_18m_u, bond_18m_d,np.nan]
price_t0 = [term_struct.loc[1.5,'price'],np.nan,np.nan]

price_tree = pd.DataFrame(price_t0)
price_tree['0.5'] = price_t6m
price_tree['1'] = price_t1

print('4.1 & 4.2')
print(price_tree)

print("Probability of upward movement at t = 1 is ", format(p_up_t1_root, '.2f'))

4.1 & 4.2
         0        0.5          1
0  96.1462  96.142586  97.530991
1      NaN  98.518379  98.728157
2      NaN        NaN  99.945015
Probability of upward movement at t = 1 is  0.79


In [14]:
print(4.3)

price =( price_t1[2] - 99) * (1- p_up) * (1-p_up_t1_root) * np.exp(-0.5*(0.0174+0.0095))

print('The option only has value if the bond price is above $99, so only in the down-down case. Risk neutral pricing gives a price of $', format(price, '.4f'))

4.3
The option only has value if the bond price is above $99, so only in the down-down case. Risk neutral pricing gives a price of $ 0.0706
